# Chapter 11: Advanced OOP Concepts

This chapter explores advanced object-oriented programming concepts in SystemVerilog that are crucial for building sophisticated verification environments and reusable components.

## Parameterized Classes

Parameterized classes allow you to create generic, reusable class templates that can be customized with different data types and parameters.

### Basic Parameterized Classes

```systemverilog
// Generic FIFO class with parameterized width and depth
class generic_fifo #(parameter WIDTH = 8, parameter DEPTH = 16);
    typedef logic [WIDTH-1:0] data_t;
    
    local data_t queue[$];
    local int max_depth;
    
    function new();
        max_depth = DEPTH;
    endfunction
    
    function void push(data_t data);
        if (queue.size() >= max_depth) begin
            $error("FIFO overflow");
            return;
        end
        queue.push_back(data);
    endfunction
    
    function data_t pop();
        if (queue.size() == 0) begin
            $error("FIFO underflow");
            return 0;
        end
        return queue.pop_front();
    endfunction
    
    function int size();
        return queue.size();
    endfunction
    
    function bit is_full();
        return (queue.size() >= max_depth);
    endfunction
    
    function bit is_empty();
        return (queue.size() == 0);
    endfunction
endclass

// Usage examples
module test_parameterized_classes;
    // 8-bit wide, 16 deep FIFO
    generic_fifo #(.WIDTH(8), .DEPTH(16)) byte_fifo;
    
    // 32-bit wide, 64 deep FIFO
    generic_fifo #(.WIDTH(32), .DEPTH(64)) word_fifo;
    
    // 16-bit wide, default depth FIFO
    generic_fifo #(.WIDTH(16)) half_word_fifo;
    
    initial begin
        byte_fifo = new();
        word_fifo = new();
        half_word_fifo = new();
        
        // Test byte FIFO
        byte_fifo.push(8'hAA);
        byte_fifo.push(8'h55);
        $display("Byte FIFO size: %0d", byte_fifo.size());
        $display("Popped: 0x%02h", byte_fifo.pop());
        
        // Test word FIFO
        word_fifo.push(32'hDEADBEEF);
        $display("Word FIFO size: %0d", word_fifo.size());
        $display("Popped: 0x%08h", word_fifo.pop());
    end
endmodule
```

### Advanced Parameterized Classes with Type Parameters

```systemverilog
// Generic container class with type parameter
class generic_container #(type T = int);
    T items[];
    int count;
    
    function new(int initial_size = 10);
        items = new[initial_size];
        count = 0;
    endfunction
    
    function void add(T item);
        if (count >= items.size()) begin
            // Resize array
            T temp[] = new[items.size() * 2];
            temp[0:items.size()-1] = items;
            items = temp;
        end
        items[count] = item;
        count++;
    endfunction
    
    function T get(int index);
        if (index >= 0 && index < count)
            return items[index];
        else begin
            $error("Index %0d out of bounds", index);
            return items[0]; // Return default
        end
    endfunction
    
    function int size();
        return count;
    endfunction
    
    function void display();
        $display("Container contents (%0d items):", count);
        for (int i = 0; i < count; i++) begin
            $display("  [%0d]: %p", i, items[i]);
        end
    endfunction
endclass

// Custom data type for testing
typedef struct {
    string name;
    int age;
    real salary;
} person_t;

module test_type_parameterized;
    // Integer container
    generic_container #(int) int_container;
    
    // String container
    generic_container #(string) string_container;
    
    // Custom type container
    generic_container #(person_t) person_container;
    
    initial begin
        int_container = new(5);
        string_container = new(3);
        person_container = new(2);
        
        // Test integer container
        int_container.add(10);
        int_container.add(20);
        int_container.add(30);
        int_container.display();
        
        // Test string container
        string_container.add("Hello");
        string_container.add("World");
        string_container.display();
        
        // Test custom type container
        person_t p1 = '{"Alice", 30, 50000.0};
        person_t p2 = '{"Bob", 25, 45000.0};
        person_container.add(p1);
        person_container.add(p2);
        person_container.display();
    end
endmodule
```

## Nested Classes

Nested classes allow you to define classes within other classes, providing better encapsulation and organization.

### Basic Nested Classes

```systemverilog
class outer_class;
    int outer_data;
    
    // Nested class definition
    class inner_class;
        int inner_data;
        
        function new(int data);
            inner_data = data;
        endfunction
        
        function void display();
            $display("Inner class data: %0d", inner_data);
        endfunction
    endclass
    
    inner_class inner_obj;
    
    function new(int outer_val, int inner_val);
        outer_data = outer_val;
        inner_obj = new(inner_val);
    endfunction
    
    function void display();
        $display("Outer class data: %0d", outer_data);
        inner_obj.display();
    endfunction
endclass

module test_nested_classes;
    outer_class obj;
    
    initial begin
        obj = new(100, 200);
        obj.display();
    end
endmodule
```

### Complex Nested Class Example - Packet with Headers

```systemverilog
class ethernet_packet;
    // Ethernet header nested class
    class eth_header;
        bit [47:0] dest_mac;
        bit [47:0] src_mac;
        bit [15:0] ether_type;
        
        function new();
            dest_mac = $random();
            src_mac = $random();
            ether_type = 16'h0800; // IP
        endfunction
        
        function void randomize_header();
            dest_mac = $random();
            src_mac = $random();
        endfunction
        
        function void display();
            $display("Ethernet Header:");
            $display("  Dest MAC: %012h", dest_mac);
            $display("  Src MAC:  %012h", src_mac);
            $display("  Type:     %04h", ether_type);
        endfunction
    endclass
    
    // IP header nested class
    class ip_header;
        bit [3:0]  version;
        bit [3:0]  header_length;
        bit [7:0]  tos;
        bit [15:0] total_length;
        bit [15:0] identification;
        bit [2:0]  flags;
        bit [12:0] fragment_offset;
        bit [7:0]  ttl;
        bit [7:0]  protocol;
        bit [15:0] checksum;
        bit [31:0] src_ip;
        bit [31:0] dest_ip;
        
        function new();
            version = 4;
            header_length = 5;
            tos = 0;
            ttl = 64;
            protocol = 8'h06; // TCP
            src_ip = $random();
            dest_ip = $random();
        endfunction
        
        function void display();
            $display("IP Header:");
            $display("  Version: %0d", version);
            $display("  Src IP:  %0d.%0d.%0d.%0d", 
                    src_ip[31:24], src_ip[23:16], src_ip[15:8], src_ip[7:0]);
            $display("  Dest IP: %0d.%0d.%0d.%0d", 
                    dest_ip[31:24], dest_ip[23:16], dest_ip[15:8], dest_ip[7:0]);
            $display("  Protocol: %02h", protocol);
        endfunction
    endclass
    
    eth_header eth_hdr;
    ip_header ip_hdr;
    bit [7:0] payload[];
    
    function new(int payload_size = 64);
        eth_hdr = new();
        ip_hdr = new(); 
        payload = new[payload_size];
        
        // Initialize payload with random data
        foreach(payload[i]) begin
            payload[i] = $random();
        end
    endfunction
    
    function void display();
        $display("=== Ethernet Packet ===");
        eth_hdr.display();
        ip_hdr.display();
        $display("Payload size: %0d bytes", payload.size());
    endfunction
    
    function void randomize_packet();
        eth_hdr.randomize_header();
        ip_hdr.src_ip = $random();
        ip_hdr.dest_ip = $random();
    endfunction
endclass
```

## Copy Constructors

Copy constructors provide a way to create copies of objects, which is essential for proper object management in verification environments.

### Basic Copy Constructor Implementation

```systemverilog
class transaction;
    int id;
    string name;
    bit [31:0] data[];
    real timestamp;
    
    function new(int _id = 0, string _name = "default");
        id = _id;
        name = _name;
        timestamp = $realtime;
        data = new[8]; // Default size
        
        // Initialize with random data
        foreach(data[i]) begin
            data[i] = $random();
        end
    endfunction
    
    // Copy constructor
    function new_copy(transaction original);
        if (original == null) begin
            $error("Cannot copy null transaction");
            return;
        end
        
        this.id = original.id;
        this.name = original.name;
        this.timestamp = $realtime; // New timestamp
        
        // Deep copy of dynamic array
        this.data = new[original.data.size()];
        this.data = original.data; // Copy contents
    endfunction
    
    // Alternative copy method
    function transaction copy();
        transaction new_trans = new();
        new_trans.new_copy(this);
        return new_trans;
    endfunction
    
    function void display();
        $display("Transaction ID: %0d, Name: %s", id, name);
        $display("Timestamp: %0t", timestamp);
        $display("Data size: %0d", data.size());
        $write("Data: ");
        foreach(data[i]) begin
            $write("%08h ", data[i]);
        end
        $display("");
    endfunction
    
    function void modify_data(int index, bit [31:0] value);
        if (index >= 0 && index < data.size()) begin
            data[index] = value;
        end
    endfunction
endclass

module test_copy_constructor;
    transaction original, copy1, copy2;
    
    initial begin
        // Create original transaction
        original = new(1, "original_trans");
        $display("=== Original Transaction ===");
        original.display();
        
        // Create copy using copy constructor
        copy1 = new();
        copy1.new_copy(original);
        $display("\n=== Copy 1 (using copy constructor) ===");
        copy1.display();
        
        // Create copy using copy method
        copy2 = original.copy();
        $display("\n=== Copy 2 (using copy method) ===");
        copy2.display();
        
        // Modify original and show independence
        original.modify_data(0, 32'hDEADBEEF);
        original.name = "modified_original";
        
        $display("\n=== After modifying original ===");
        $display("Original:");
        original.display();
        $display("Copy 1 (should be unchanged):");
        copy1.display();
        $display("Copy 2 (should be unchanged):");
        copy2.display();
    end
endmodule
```

## Shallow vs. Deep Copy

Understanding the difference between shallow and deep copying is crucial for proper object management.

### Demonstrating Shallow vs. Deep Copy

```systemverilog
class inner_data;
    int value;
    string tag;
    
    function new(int v = 0, string t = "default");
        value = v;
        tag = t;
    endfunction
    
    function void display();
        $display("    Inner data - Value: %0d, Tag: %s", value, tag);
    endfunction
    
    function inner_data copy();
        inner_data new_inner = new(this.value, this.tag);
        return new_inner;
    endfunction
endclass

class container;
    int id;
    inner_data inner_obj;
    int simple_array[];
    
    function new(int _id = 0);
        id = _id;
        inner_obj = new(100, "inner");
        simple_array = new[5];
        
        foreach(simple_array[i]) begin
            simple_array[i] = i * 10;
        end
    endfunction
    
    // Shallow copy - copies references, not objects
    function container shallow_copy();
        container new_container = new();
        new_container.id = this.id;
        new_container.inner_obj = this.inner_obj; // Same reference!
        new_container.simple_array = this.simple_array; // Same reference!
        return new_container;
    endfunction
    
    // Deep copy - creates new objects
    function container deep_copy();
        container new_container = new();
        new_container.id = this.id;
        
        // Create new inner object
        new_container.inner_obj = this.inner_obj.copy();
        
        // Create new array and copy contents
        new_container.simple_array = new[this.simple_array.size()];
        new_container.simple_array = this.simple_array;
        
        return new_container;
    endfunction
    
    function void display();
        $display("Container ID: %0d", id);
        inner_obj.display();
        $write("  Array: ");
        foreach(simple_array[i]) begin
            $write("%0d ", simple_array[i]);
        end
        $display("");
    endfunction
    
    function void modify_inner(int new_value, string new_tag);
        inner_obj.value = new_value;
        inner_obj.tag = new_tag;
    endfunction
    
    function void modify_array(int index, int value);
        if (index >= 0 && index < simple_array.size()) begin
            simple_array[index] = value;
        end
    endfunction
endclass

module test_shallow_vs_deep_copy;
    container original, shallow_copy, deep_copy;
    
    initial begin
        // Create original
        original = new(1);
        $display("=== Original Container ===");
        original.display();
        
        // Create shallow copy
        shallow_copy = original.shallow_copy();
        $display("\n=== Shallow Copy ===");
        shallow_copy.display();
        
        // Create deep copy
        deep_copy = original.deep_copy();
        $display("\n=== Deep Copy ===");
        deep_copy.display();
        
        $display("\n=== Modifying Original ===");
        original.modify_inner(999, "modified");
        original.modify_array(0, 555);
        
        $display("Original after modification:");
        original.display();
        
        $display("Shallow copy (affected by modification):");
        shallow_copy.display();
        
        $display("Deep copy (unaffected by modification):");
        deep_copy.display();
        
        // Demonstrate the problem with shallow copy
        $display("\n=== The Shallow Copy Problem ===");
        $display("Shallow copy shares the same inner object and array!");
        $display("Original inner_obj handle: %p", original.inner_obj);
        $display("Shallow copy inner_obj handle: %p", shallow_copy.inner_obj);
        $display("Deep copy inner_obj handle: %p", deep_copy.inner_obj);
        
        $display("Handles are %s", 
                (original.inner_obj == shallow_copy.inner_obj) ? "SAME" : "DIFFERENT");
    end
endmodule
```

## Class Handles and References

Understanding class handles and references is fundamental to working with objects in SystemVerilog.

### Class Handle Basics

```systemverilog
class simple_class;
    int data;
    string name;
    
    function new(int d = 0, string n = "default");
        data = d;
        name = n;
    endfunction
    
    function void display();
        $display("Object: %s, Data: %0d, Handle: %p", name, data, this);
    endfunction
endclass

module test_class_handles;
    simple_class obj1, obj2, obj3;
    
    initial begin
        $display("=== Class Handle Demonstration ===");
        
        // Initially all handles are null
        $display("Initial handle values:");
        $display("obj1: %p", obj1);
        $display("obj2: %p", obj2);
        $display("obj3: %p", obj3);
        
        // Create objects
        obj1 = new(10, "first");
        obj2 = new(20, "second");
        
        $display("\nAfter creating objects:");
        obj1.display();
        obj2.display();
        
        // Handle assignment (not copying the object!)
        obj3 = obj1;
        $display("\nAfter obj3 = obj1:");
        obj1.display();
        obj3.display();
        
        // Modifying through one handle affects the other
        obj3.data = 999;
        obj3.name = "modified";
        
        $display("\nAfter modifying through obj3:");
        obj1.display();
        obj3.display();
        
        // Handle comparison
        $display("\nHandle comparisons:");
        $display("obj1 == obj2: %b", obj1 == obj2);
        $display("obj1 == obj3: %b", obj1 == obj3);
        $display("obj1 === obj3: %b", obj1 === obj3);
        
        // Setting handle to null
        obj1 = null;
        $display("\nAfter obj1 = null:");
        $display("obj1: %p", obj1);
        $display("obj3: %p", obj3);
        obj3.display(); // Still works because object exists
    end
endmodule
```

### Advanced Handle Management

```systemverilog
class managed_object;
    static int object_count = 0;
    int object_id;
    string name;
    
    function new(string n = "unnamed");
        object_count++;
        object_id = object_count;
        name = n;
        $display("Created object %0d: %s", object_id, name);
    endfunction
    
    function void display();
        $display("Object %0d (%s) - Handle: %p", object_id, name, this);
    endfunction
    
    static function int get_object_count();
        return object_count;
    endfunction
endclass

class handle_manager;
    managed_object objects[];
    int count;
    
    function new(int max_objects = 10);
        objects = new[max_objects];
        count = 0;
    endfunction
    
    function int add_object(managed_object obj);
        if (count >= objects.size()) begin
            $display("Handle manager full!");
            return -1;
        end
        
        objects[count] = obj;
        count++;
        return count - 1;
    endfunction
    
    function managed_object get_object(int index);
        if (index >= 0 && index < count) begin
            return objects[index];
        end
        return null;
    endfunction
    
    function void remove_object(int index);
        if (index >= 0 && index < count) begin
            // Shift remaining objects
            for (int i = index; i < count - 1; i++) begin
                objects[i] = objects[i + 1];
            end
            objects[count - 1] = null;
            count--;
        end
    endfunction
    
    function void display_all();
        $display("Handle Manager contains %0d objects:", count);
        for (int i = 0; i < count; i++) begin
            if (objects[i] != null) begin
                $write("  [%0d]: ", i);
                objects[i].display();
            end
        end
    endfunction
    
    function void cleanup();
        for (int i = 0; i < objects.size(); i++) begin
            objects[i] = null;
        end
        count = 0;
        $display("Handle manager cleaned up");
    endfunction
endclass

module test_handle_management;
    handle_manager manager;
    managed_object obj1, obj2, obj3, obj4;
    
    initial begin
        $display("=== Handle Management Test ===");
        
        manager = new(5);
        
        // Create and add objects
        obj1 = new("Alpha");
        obj2 = new("Beta");
        obj3 = new("Gamma");
        
        manager.add_object(obj1);
        manager.add_object(obj2);
        manager.add_object(obj3);
        
        $display("\nTotal objects created: %0d", 
                managed_object::get_object_count());
        
        manager.display_all();
        
        // Get object by index
        obj4 = manager.get_object(1);
        if (obj4 != null) begin
            $display("\nRetrieved object at index 1:");
            obj4.display();
        end
        
        // Remove object
        $display("\nRemoving object at index 1:");
        manager.remove_object(1);
        manager.display_all();
        
        // Cleanup
        $display("\nCleaning up manager:");
        manager.cleanup();
        manager.display_all();
        
        $display("\nFinal object count: %0d", 
                managed_object::get_object_count());
    end
endmodule
```

### Reference Counting and Smart Pointers Pattern

```systemverilog
class ref_counted_object;
    static int total_objects = 0;
    int object_id;
    string data;
    int ref_count;
    
    function new(string d = "default");
        total_objects++;
        object_id = total_objects;
        data = d;
        ref_count = 1; // Creator gets first reference
        $display("Created object %0d with data '%s'", object_id, data);
    endfunction
    
    function void add_ref();
        ref_count++;
        $display("Object %0d ref count increased to %0d", object_id, ref_count);
    endfunction
    
    function void release_ref();
        ref_count--;
        $display("Object %0d ref count decreased to %0d", object_id, ref_count);
        if (ref_count <= 0) begin
            $display("Object %0d being destroyed", object_id);
            // In real implementation, this would trigger cleanup
        end
    endfunction
    
    function int get_ref_count();
        return ref_count;
    endfunction
    
    function void display();
        $display("Object %0d: '%s' (refs: %0d)", object_id, data, ref_count);
    endfunction
endclass

class smart_pointer;
    ref_counted_object obj;
    
    function new(ref_counted_object o = null);
        if (o != null) begin
            obj = o;
            obj.add_ref();
        end
    endfunction
    
    function void assign(ref_counted_object o);
        // Release current object
        if (obj != null) begin
            obj.release_ref();
        end
        
        // Assign new object
        obj = o;
        if (obj != null) begin
            obj.add_ref();
        end
    endfunction
    
    function ref_counted_object get();
        return obj;
    endfunction
    
    function void reset();
        if (obj != null) begin
            obj.release_ref();
            obj = null;
        end
    endfunction
    
    // Destructor simulation
    function void cleanup();
        reset();
    endfunction
endclass

module test_reference_counting;
    ref_counted_object obj1;
    smart_pointer ptr1, ptr2, ptr3;
    
    initial begin
        $display("=== Reference Counting Test ===");
        
        // Create object
        obj1 = new("Test Data");
        obj1.display();
        
        // Create smart pointers
        ptr1 = new(obj1);
        ptr2 = new();
        ptr3 = new();
        
        obj1.display();
        
        // Assign to other pointers
        ptr2.assign(obj1);
        ptr3.assign(obj1);
        
        obj1.display();
        
        // Release one pointer
        $display("\nReleasing ptr1:");
        ptr1.reset();
        obj1.display();
        
        // Release another pointer
        $display("\nReleasing ptr2:");
        ptr2.cleanup();
        obj1.display();
        
        // Release last pointer
        $display("\nReleasing ptr3:");
        ptr3.cleanup();
        obj1.display();
        
        // Release original reference
        $display("\nReleasing original reference:");
        obj1.release_ref();
    end
endmodule
```

## Best Practices and Common Patterns

### Factory Pattern with Parameterized Classes

```systemverilog
// Base transaction class
virtual class base_transaction;
    int id;
    real timestamp;
    
    function new(int _id = 0);
        id = _id;
        timestamp = $realtime;
    endfunction
    
    pure virtual function void display();
    pure virtual function base_transaction copy();
endclass

// Specific transaction types
class read_transaction extends base_transaction;
    bit [31:0] address;
    
    function new(int _id = 0, bit [31:0] addr = 0);
        super.new(_id);
        address = addr;
    endfunction
    
    virtual function void display();
        $display("Read Transaction - ID: %0d, Addr: 0x%08h, Time: %0t", 
                id, address, timestamp);
    endfunction
    
    virtual function base_transaction copy();
        read_transaction new_trans = new(this.id, this.address);
        return new_trans;
    endfunction
endclass

class write_transaction extends base_transaction;
    bit [31:0] address;
    bit [31:0] data;
    
    function new(int _id = 0, bit [31:0] addr = 0, bit [31:0] d = 0);
        super.new(_id);
        address = addr;
        data = d;
    endfunction
    
    virtual function void display();
        $display("Write Transaction - ID: %0d, Addr: 0x%08h, Data: 0x%08h, Time: %0t", 
                id, address, data, timestamp);
    endfunction
    
    virtual function base_transaction copy();
        write_transaction new_trans = new(this.id, this.address, this.data);
        return new_trans;
    endfunction
endclass

// Transaction factory
class transaction_factory #(type T = base_transaction);
    static int next_id = 1;
    
    static function T create();
        T trans = new(next_id++);
        return trans;
    endfunction
    
    static function T create_with_id(int id);
        T trans = new(id);
        return trans;
    endfunction
endclass

module test_factory_pattern;
    // Specialized factories
    transaction_factory #(read_transaction) read_factory;
    transaction_factory #(write_transaction) write_factory;
    
    read_transaction rd_trans;
    write_transaction wr_trans;
    base_transaction trans_array[];
    
    initial begin
        $display("=== Factory Pattern Test ===");
        
        trans_array = new[4];
        
        // Create transactions using factories
        rd_trans = read_factory::create();
        rd_trans.address = 32'h1000;
        trans_array[0] = rd_trans;
        
        wr_trans = write_factory::create();
        wr_trans.address = 32'h2000;
        wr_trans.data = 32'hDEADBEEF;
        trans_array[1] = wr_trans;
        
        rd_trans = read_factory::create();
        rd_trans.address = 32'h3000;
        trans_array[2] = rd_trans;
        
        wr_trans = write_factory::create();
        wr_trans.address = 32'h4000;
        wr_trans.data = 32'hCAFEBABE;
        trans_array[3] = wr_trans;
        
        // Display all transactions
        foreach(trans_array[i]) begin
            trans_array[i].display();
        end
        
        // Test copying
        $display("\n=== Testing Copy ===");
        base_transaction copied = trans_array[1].copy();
        copied.display();
    end
endmodule
```

# SystemVerilog Advanced OOP - Simple Examples by Topic

## Parameterized Classes

### **Example 1:** Simple Generic Stack
**Description:** Basic Stack class with parameterized depth and data width. Shows push(), pop(), and size() operations with different configurations.

In [4]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_1__generic_stack/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// generic_stack.sv
package stack_pkg;

  // Generic stack class with parameterized depth and data width
  class generic_stack #(
    parameter int DEPTH = 8,
    parameter int DATA_WIDTH = 32
  );
    
    // Stack storage array
    logic [DATA_WIDTH-1:0] stack_data[DEPTH];
    
    // Stack pointer (points to next free location)
    int stack_pointer;
    
    // Constructor
    function new();
      stack_pointer = 0;
      $display("Stack created: depth=%0d, data_width=%0d", 
               DEPTH, DATA_WIDTH);
    endfunction
    
    // Push data onto stack
    function bit push(logic [DATA_WIDTH-1:0] data);
      if (stack_pointer >= DEPTH) begin
        $display("ERROR: Stack overflow! Cannot push 0x%0h", data);
        return 0;
      end
      
      stack_data[stack_pointer] = data;
      stack_pointer++;
      $display("Pushed: 0x%0h (stack size: %0d)", data, stack_pointer);
      return 1;
    endfunction
    
    // Pop data from stack
    function bit pop(output logic [DATA_WIDTH-1:0] data);
      if (stack_pointer == 0) begin
        $display("ERROR: Stack underflow! Cannot pop from empty stack");
        return 0;
      end
      
      stack_pointer--;
      data = stack_data[stack_pointer];
      $display("Popped: 0x%0h (stack size: %0d)", data, stack_pointer);
      return 1;
    endfunction
    
    // Get current stack size
    function int size();
      return stack_pointer;
    endfunction
    
    // Check if stack is empty
    function bit is_empty();
      return (stack_pointer == 0);
    endfunction
    
    // Check if stack is full
    function bit is_full();
      return (stack_pointer >= DEPTH);
    endfunction
    
    // Display stack contents
    function void display();
      $display("Stack contents (size: %0d/%0d):", stack_pointer, DEPTH);
      if (stack_pointer == 0) begin
        $display("  [empty]");
      end else begin
        for (int i = stack_pointer - 1; i >= 0; i--) begin
          $display("  [%0d]: 0x%0h", i, stack_data[i]);
        end
      end
    endfunction
    
  endclass

endpackage

// Top-level module for design under test
module generic_stack_design();
  import stack_pkg::*;
  
  // Create different stack instances
  generic_stack #(.DEPTH(4), .DATA_WIDTH(8))  byte_stack;
  generic_stack #(.DEPTH(8), .DATA_WIDTH(16)) word_stack;
  generic_stack #(.DEPTH(16), .DATA_WIDTH(32)) dword_stack;
  
  initial begin
    $display("=== Generic Stack Design Demo ===");
    $display();
    
    // Create stack instances
    byte_stack = new();
    word_stack = new();
    dword_stack = new();
    
    $display();
    $display("Stack instances created successfully");
  end
  
endmodule
```

```systemverilog
// generic_stack_testbench.sv
module generic_stack_test_bench;
  import stack_pkg::*;
  
  // Instantiate design under test
  generic_stack_design DESIGN_INSTANCE();
  
  // Test variables
  generic_stack #(.DEPTH(4), .DATA_WIDTH(8)) test_stack;
  logic [7:0] popped_data;
  bit success;
  
  initial begin
    // Setup wave dumping
    $dumpfile("generic_stack_test_bench.vcd");
    $dumpvars(0, generic_stack_test_bench);
    
    $display("=== Generic Stack Testbench ===");
    $display();
    
    // Create test stack
    test_stack = new();
    $display();
    
    // Test 1: Push operations
    $display("--- Test 1: Push Operations ---");
    success = test_stack.push(8'hAA);
    success = test_stack.push(8'hBB);
    success = test_stack.push(8'hCC);
    test_stack.display();
    $display();
    
    // Test 2: Pop operations
    $display("--- Test 2: Pop Operations ---");
    success = test_stack.pop(popped_data);
    success = test_stack.pop(popped_data);
    test_stack.display();
    $display();
    
    // Test 3: Stack overflow
    $display("--- Test 3: Stack Overflow Test ---");
    success = test_stack.push(8'h11);
    success = test_stack.push(8'h22);
    success = test_stack.push(8'h33);
    success = test_stack.push(8'h44);  // This should cause overflow
    test_stack.display();
    $display();
    
    // Test 4: Stack underflow
    $display("--- Test 4: Stack Underflow Test ---");
    // Empty the stack first
    while (!test_stack.is_empty()) begin
      success = test_stack.pop(popped_data);
    end
    // Try to pop from empty stack
    success = test_stack.pop(popped_data);  // This should cause underflow
    test_stack.display();
    $display();
    
    // Test 5: Different stack sizes
    $display("--- Test 5: Different Stack Configurations ---");
    begin
      generic_stack #(.DEPTH(2), .DATA_WIDTH(16)) small_stack;
      generic_stack #(.DEPTH(8), .DATA_WIDTH(32)) large_stack;
      
      small_stack = new();
      large_stack = new();
      
      // Test small stack
      success = small_stack.push(16'h1234);
      success = small_stack.push(16'h5678);
      small_stack.display();
      
      // Test large stack
      success = large_stack.push(32'hDEADBEEF);
      success = large_stack.push(32'hCAFEBABE);
      large_stack.display();
    end
    $display();
    
    // Test 6: Status functions
    $display("--- Test 6: Status Functions ---");
    $display("Stack size: %0d", test_stack.size());
    $display("Is empty: %0b", test_stack.is_empty());
    $display("Is full: %0b", test_stack.is_full());
    
    success = test_stack.push(8'hFF);
    $display("After push - size: %0d, empty: %0b, full: %0b", 
             test_stack.size(), test_stack.is_empty(), 
             test_stack.is_full());
    
    $display();
    $display("=== Testbench Complete ===");
    
    #10;  // Wait a bit for wave dump
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Generic Stack Design Demo ===

Stack created: depth=4, data_width=8
Stack created: depth=8, data_width=16
Stack created: depth=16, data_width=32

Stack instances created successfully
=== Generic Stack Testbench ===

Stack created: depth=4, data_width=8

--- Test 1: Push Operations ---
Pushed: 0xaa (stack size: 1)
Pushed: 0xbb (stack size: 2)
Pushed: 0xcc (stack size: 3)
Stack contents (size: 3/4):
  [2]: 0xcc
  [1]: 0xbb
  [0]: 0xaa

--- Test 2: Pop Operations ---
Popped: 0xcc (stack size: 2)
Popped: 0xbb (stack size: 1)
Stack contents (size: 1/4):
  [0]: 0xaa

--- Test 3: Stack Overflow Test ---
Pushed: 0x11 (stack size: 2)
Pushed: 0x22 (stack size: 3)
Pushed: 0x33 (stack size: 4)
ERROR: Stack overflow! Cannot push 0x44
Stack contents (size: 4/4):
  [3]: 0x33
  [2]: 0x22
  [1]: 0x11
  [0]: 0xaa

--- Test 4: Stack Underflow Test ---
Popped: 0x33 (stack size: 3)
Popped: 0x22 (stack size: 2)
Popped: 0x11 (stack size: 1)
Popped: 0xaa (stack size: 0)
ERROR:

0

### **Example 2:** Generic Counter
**Description:** Counter class with parameterized width and increment value. Demonstrates customizable counting behavior with overflow detection.

In [5]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_2__generic_counter/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// generic_counter.sv
`ifndef COUNTER_PKG_SV
`define COUNTER_PKG_SV

// Package containing the generic counter class
package counter_pkg;

  // Generic counter class with parameterized width and increment
  class generic_counter #(parameter int WIDTH = 8, parameter int INCREMENT = 1);
    
    // Internal counter value
    logic [WIDTH-1:0] count_value;
    logic [WIDTH-1:0] max_value;
    bit overflow_flag;
    
    // Constructor
    function new();
      count_value = 0;
      max_value = {WIDTH{1'b1}};  // All ones for maximum value
      overflow_flag = 0;
      $display("Counter created: WIDTH=%0d, INCREMENT=%0d, MAX=%0d", 
               WIDTH, INCREMENT, max_value);
    endfunction
    
    // Increment the counter
    function void increment();
      logic [WIDTH:0] temp_sum;  // Extra bit for overflow detection
      logic [WIDTH:0] extended_count;
      logic [WIDTH:0] extended_increment;
      
      /* verilator lint_off WIDTHTRUNC */
      /* verilator lint_off WIDTHEXPAND */
      extended_count = {1'b0, count_value};
      extended_increment = INCREMENT;
      temp_sum = extended_count + extended_increment;
      /* verilator lint_on WIDTHEXPAND */
      /* verilator lint_on WIDTHTRUNC */
      
      // Check for overflow
      if (temp_sum > {1'b0, max_value}) begin
        overflow_flag = 1;
        count_value = temp_sum[WIDTH-1:0];  // Wrap around
        $display("Counter overflow detected! Value wrapped to %0d", 
                 count_value);
      end else begin
        overflow_flag = 0;
        count_value = temp_sum[WIDTH-1:0];
      end
    endfunction
    
    // Reset the counter
    function void reset();
      count_value = 0;
      overflow_flag = 0;
      $display("Counter reset to 0");
    endfunction
    
    // Get current count value
    function logic [WIDTH-1:0] get_count();
      return count_value;
    endfunction
    
    // Get overflow status
    function bit get_overflow_flag();
      return overflow_flag;
    endfunction
    
    // Get maximum possible value
    function logic [WIDTH-1:0] get_max_value();
      return max_value;
    endfunction
    
    // Display current status
    function void display_status();
      $display("Counter Status - Value: %0d, Overflow: %0b, Max: %0d", 
               count_value, overflow_flag, max_value);
    endfunction
    
  endclass

endpackage

`endif // COUNTER_PKG_SV
```

```systemverilog
// generic_counter_testbench.sv
`include "generic_counter.sv"

module generic_counter_testbench;
  
  // Import the counter package
  import counter_pkg::*;
  
  // Declare counter instances with different parameters
  generic_counter #(.WIDTH(4), .INCREMENT(1)) counter_4bit_inc1;
  generic_counter #(.WIDTH(8), .INCREMENT(3)) counter_8bit_inc3;
  generic_counter #(.WIDTH(6), .INCREMENT(7)) counter_6bit_inc7;
  
  initial begin
    // Set up waveform dumping
    $dumpfile("generic_counter_testbench.vcd");
    $dumpvars(0, generic_counter_testbench);
    
    $display("=== Generic Counter Test Starting ===");
    $display();
    
    // Create counter instances
    counter_4bit_inc1 = new();
    counter_8bit_inc3 = new();
    counter_6bit_inc7 = new();
    
    $display();
    $display("=== Testing 4-bit Counter (increment=1) ===");
    test_4bit_counter();
    
    $display();
    $display("=== Testing 8-bit Counter (increment=3) ===");
    test_8bit_counter();
    
    $display();
    $display("=== Testing 6-bit Counter (increment=7) ===");
    test_6bit_counter();
    
    $display();
    $display("=== All Tests Complete ===");
    $finish;
  end
  
  // Task to test 4-bit counter
  task test_4bit_counter();
    $display("Testing 4-bit counter basic functionality:");
    
    // Display initial status
    counter_4bit_inc1.display_status();
    
    // Increment several times
    for (int i = 0; i < 5; i++) begin
      counter_4bit_inc1.increment();
      $display("After increment %0d: count = %0d, overflow = %0b", 
               i+1, counter_4bit_inc1.get_count(), 
               counter_4bit_inc1.get_overflow_flag());
    end
    
    // Reset and verify
    counter_4bit_inc1.reset();
    counter_4bit_inc1.display_status();
    
  endtask
  
  // Task to test 8-bit counter
  task test_8bit_counter();
    $display("Testing 8-bit counter basic functionality:");
    
    // Display initial status
    counter_8bit_inc3.display_status();
    
    // Increment several times
    for (int i = 0; i < 5; i++) begin
      counter_8bit_inc3.increment();
      $display("After increment %0d: count = %0d, overflow = %0b", 
               i+1, counter_8bit_inc3.get_count(), 
               counter_8bit_inc3.get_overflow_flag());
    end
    
    // Reset and verify
    counter_8bit_inc3.reset();
    counter_8bit_inc3.display_status();
    
  endtask
  
  // Task to test 6-bit counter overflow behavior
  task test_6bit_counter();
    int max_val;
    int increment_count;
    
    $display("Testing 6-bit counter overflow behavior:");
    
    /* verilator lint_off WIDTHEXPAND */
    max_val = int'(counter_6bit_inc7.get_max_value());
    /* verilator lint_on WIDTHEXPAND */
    increment_count = 0;
    
    // Increment until overflow occurs
    while (!counter_6bit_inc7.get_overflow_flag() && increment_count < 20) begin
      counter_6bit_inc7.increment();
      increment_count++;
      $display("Increment %0d: count = %0d (max = %0d)", 
               increment_count, counter_6bit_inc7.get_count(), max_val);
    end
    
    // Continue a few more increments to show wrap-around behavior
    if (counter_6bit_inc7.get_overflow_flag()) begin
      $display("Overflow detected! Continuing to show wrap-around...");
      for (int i = 0; i < 3; i++) begin
        counter_6bit_inc7.increment();
        $display("After overflow increment %0d: count = %0d", 
                 i+1, counter_6bit_inc7.get_count());
      end
    end
    
    counter_6bit_inc7.display_status();
    
  endtask

endmodule
```

Verilator Simulation Output:

Counter created: WIDTH=4, INCREMENT=1, MAX=15
Counter created: WIDTH=8, INCREMENT=3, MAX=255
Counter created: WIDTH=6, INCREMENT=7, MAX=63

=== Testing 4-bit Counter (increment=1) ===
Testing 4-bit counter basic functionality:
Counter Status - Value: 0, Overflow: 0, Max: 15
After increment 1: count = 1, overflow = 0
After increment 2: count = 2, overflow = 0
After increment 3: count = 3, overflow = 0
After increment 4: count = 4, overflow = 0
After increment 5: count = 5, overflow = 0
Counter reset to 0
Counter Status - Value: 0, Overflow: 0, Max: 15

=== Testing 8-bit Counter (increment=3) ===
Testing 8-bit counter basic functionality:
Counter Status - Value: 0, Overflow: 0, Max: 255
After increment 1: count = 3, overflow = 0
After increment 2: count = 6, overflow = 0
After increment 3: count = 9, overflow = 0
After increment 4: count = 12, overflow = 0
After increment 5: count = 15, overflow = 0
Counter reset to 0
Counter Status - Value: 0, Overflow: 0, 

0

### **Example 3:** Configurable Buffer
**Description:** Simple Buffer class with parameterized size and data type. Shows basic read/write operations with different buffer sizes.

In [9]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_3__configurable_buffer/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// configurable_buffer_design.sv
package buffer_pkg;

  // Generic configurable buffer class
  class configurable_buffer #(
    parameter int BUFFER_SIZE = 8,
    parameter type DATA_TYPE = logic [7:0]
  );
    
    // Buffer storage array
    protected DATA_TYPE buffer_memory[];
    protected int write_pointer;
    protected int read_pointer;
    protected int item_count;
    
    // Constructor
    function new();
      buffer_memory = new[BUFFER_SIZE];
      write_pointer = 0;
      read_pointer = 0;
      item_count = 0;
      $display("Buffer created: size=%0d, data_width=%0d bits", 
               BUFFER_SIZE, $bits(DATA_TYPE));
    endfunction
    
    // Write data to buffer
    function bit write_data(DATA_TYPE data);
      if (is_full()) begin
        $display("ERROR: Buffer full! Cannot write 0x%0h", data);
        return 0;
      end
      
      buffer_memory[write_pointer] = data;
      write_pointer = (write_pointer + 1) % BUFFER_SIZE;
      item_count++;
      $display("Written: 0x%0h at position %0d (count=%0d)", 
               data, write_pointer-1, item_count);
      return 1;
    endfunction
    
    // Read data from buffer
    function bit read_data(ref DATA_TYPE data);
      if (is_empty()) begin
        $display("ERROR: Buffer empty! Cannot read");
        return 0;
      end
      
      data = buffer_memory[read_pointer];
      read_pointer = (read_pointer + 1) % BUFFER_SIZE;
      item_count--;
      $display("Read: 0x%0h from position %0d (count=%0d)", 
               data, read_pointer-1, item_count);
      return 1;
    endfunction
    
    // Check if buffer is full
    function bit is_full();
      return (item_count == BUFFER_SIZE);
    endfunction
    
    // Check if buffer is empty
    function bit is_empty();
      return (item_count == 0);
    endfunction
    
    // Get current item count
    function int get_count();
      return item_count;
    endfunction
    
    // Display buffer status
    function void display_status();
      $display("Buffer Status: %0d/%0d items, WP=%0d, RP=%0d", 
               item_count, BUFFER_SIZE, write_pointer, read_pointer);
    endfunction
    
    // Reset buffer
    function void reset();
      write_pointer = 0;
      read_pointer = 0;
      item_count = 0;
      $display("Buffer reset");
    endfunction
    
  endclass

endpackage

// Top-level design module
module configurable_buffer_design;
  import buffer_pkg::*;
  
  // No actual hardware - just a placeholder for the package
  initial begin
    $display("Configurable Buffer Design Module Loaded");
  end
  
endmodule
```

```systemverilog
// configurable_buffer_design_testbench.sv
module configurable_buffer_testbench;
  import buffer_pkg::*;
  
  // Instantiate design under test
  configurable_buffer_design DESIGN_INSTANCE_NAME();
  
  // Test different buffer configurations
  configurable_buffer #(.BUFFER_SIZE(4), .DATA_TYPE(logic [7:0])) 
    byte_buffer;
  
  configurable_buffer #(.BUFFER_SIZE(8), .DATA_TYPE(logic [15:0])) 
    word_buffer;
  
  configurable_buffer #(.BUFFER_SIZE(16), .DATA_TYPE(logic [31:0])) 
    dword_buffer;
  
  // Test variables
  logic [7:0]  byte_data;
  logic [15:0] word_data;
  logic [31:0] dword_data;
  bit success;
  
  initial begin
    // Dump waves for verilator
    $dumpfile("configurable_buffer_testbench.vcd");
    $dumpvars(0, configurable_buffer_testbench);
    
    $display("=== Configurable Buffer Testbench Starting ===");
    $display();
    
    // Create buffer instances
    byte_buffer = new();
    word_buffer = new();
    dword_buffer = new();
    
    $display("\n=== Testing 8-bit Buffer (size=4) ===");
    test_byte_buffer();
    
    $display("\n=== Testing 16-bit Buffer (size=8) ===");
    test_word_buffer();
    
    $display("\n=== Testing 32-bit Buffer (size=16) ===");
    test_dword_buffer();
    
    $display("\n=== Testing Buffer Overflow/Underflow ===");
    test_edge_cases();
    
    #10; // Wait for a few time units
    $display("\n=== Configurable Buffer Testbench Complete ===");
    $finish;
  end
  
  // Test 8-bit buffer
  task test_byte_buffer();
    $display("Testing byte buffer operations...");
    
    // Fill buffer partially
    for (int i = 0; i < 3; i++) begin
      byte_data = 8'h10 + 8'(i);
      success = byte_buffer.write_data(byte_data);
      byte_buffer.display_status();
    end
    
    // Read some data
    for (int i = 0; i < 2; i++) begin
      success = byte_buffer.read_data(byte_data);
      byte_buffer.display_status();
    end
    
    // Write more data
    byte_data = 8'hFF;
    success = byte_buffer.write_data(byte_data);
    byte_buffer.display_status();
  endtask
  
  // Test 16-bit buffer
  task test_word_buffer();
    $display("Testing word buffer operations...");
    
    // Fill buffer with pattern
    for (int i = 0; i < 5; i++) begin
      word_data = 16'h1000 + 16'(i * 16'h0111);
      success = word_buffer.write_data(word_data);
      if (i % 2 == 0) word_buffer.display_status();
    end
    
    // Read all data
    while (!word_buffer.is_empty()) begin
      success = word_buffer.read_data(word_data);
      if (word_buffer.get_count() % 2 == 0) word_buffer.display_status();
    end
  endtask
  
  // Test 32-bit buffer
  task test_dword_buffer();
    $display("Testing dword buffer operations...");
    
    // Write pattern data
    for (int i = 0; i < 10; i++) begin
      dword_data = 32'h12345000 + 32'(i);
      success = dword_buffer.write_data(dword_data);
      if (i % 3 == 0) dword_buffer.display_status();
    end
    
    // Read half the data
    for (int i = 0; i < 5; i++) begin
      success = dword_buffer.read_data(dword_data);
      if (i % 2 == 0) dword_buffer.display_status();
    end
    
    // Reset and verify
    dword_buffer.reset();
    dword_buffer.display_status();
  endtask
  
  // Test edge cases
  task test_edge_cases();
    $display("Testing overflow and underflow conditions...");
    
    // Reset byte buffer for clean test
    byte_buffer.reset();
    
    // Fill buffer completely
    $display("Filling buffer to capacity...");
    for (int i = 0; i < 5; i++) begin  // Try to write 5 items to 4-slot buffer
      byte_data = 8'hA0 + 8'(i);
      success = byte_buffer.write_data(byte_data);
      byte_buffer.display_status();
    end
    
    // Try to read more than available
    $display("Emptying buffer and trying to read beyond...");
    for (int i = 0; i < 6; i++) begin  // Try to read 6 items from 4-slot buffer
      success = byte_buffer.read_data(byte_data);
      byte_buffer.display_status();
    end
  endtask
  
endmodule
```

Verilator Simulation Output:
Configurable Buffer Design Module Loaded
=== Configurable Buffer Testbench Starting ===

Buffer created: size=4, data_width=8 bits
Buffer created: size=8, data_width=16 bits
Buffer created: size=16, data_width=32 bits

=== Testing 8-bit Buffer (size=4) ===
Testing byte buffer operations...
Written: 0x10 at position 0 (count=1)
Buffer Status: 1/4 items, WP=1, RP=0
Written: 0x11 at position 1 (count=2)
Buffer Status: 2/4 items, WP=2, RP=0
Written: 0x12 at position 2 (count=3)
Buffer Status: 3/4 items, WP=3, RP=0
Read: 0x10 from position 0 (count=2)
Buffer Status: 2/4 items, WP=3, RP=1
Read: 0x11 from position 1 (count=1)
Buffer Status: 1/4 items, WP=3, RP=2
Written: 0xff at position -1 (count=2)
Buffer Status: 2/4 items, WP=0, RP=2

=== Testing 16-bit Buffer (size=8) ===
Testing word buffer operations...
Written: 0x1000 at position 0 (count=1)
Buffer Status: 1/8 items, WP=1, RP=0
Written: 0x1111 at position 1 (count=2)
Written: 0x1222 at position 2 (count=3)


0

### 4. Generic Register Bank
**Description:** Register bank with parameterized number of registers and register width. Basic read/write functionality for different configurations.

In [21]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_4__generic_register_bank/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// generic_register_bank.sv
module generic_register_bank #(
  parameter int unsigned NUM_REGISTERS = 16,
  parameter int unsigned DATA_WIDTH = 32,
  parameter int unsigned ADDR_WIDTH = $clog2(NUM_REGISTERS)
) (
  input  logic                    clk,
  input  logic                    rst_n,
  input  logic                    write_enable,
  input  logic [ADDR_WIDTH-1:0]   address,
  input  logic [DATA_WIDTH-1:0]   write_data,
  output logic [DATA_WIDTH-1:0]   read_data,
  output logic                    valid_address
);

  // Register bank storage
  logic [DATA_WIDTH-1:0] registers [NUM_REGISTERS-1:0];
  
  // Address validation - compare with proper width extension
  assign valid_address = ({{32-ADDR_WIDTH{1'b0}}, address} < NUM_REGISTERS);
  
  // Read operation (combinatorial)
  always_comb begin
    if (valid_address) begin
      read_data = registers[address];
    end else begin
      read_data = '0;  // Return zero for invalid addresses
    end
  end
  
  // Write operation (synchronous)
  always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
      // Reset all registers to zero
      for (int i = 0; i < NUM_REGISTERS; i++) begin
        registers[i] <= '0;
      end
    end else if (write_enable && valid_address) begin
      registers[address] <= write_data;
    end
  end
  
  // Display register contents (for debugging)
  initial begin
    $display("Generic Register Bank instantiated:");
    $display("  NUM_REGISTERS = %0d", NUM_REGISTERS);
    $display("  DATA_WIDTH = %0d", DATA_WIDTH);
    $display("  ADDR_WIDTH = %0d", ADDR_WIDTH);
  end

endmodule
```

```systemverilog
// generic_register_bank_testbench.sv
`include "register_bank_pkg.sv"

module register_bank_test_module;
  import register_bank_pkg::*;
  
  // Test parameters for different configurations
  parameter int unsigned NUM_REGS_SMALL = 8;
  parameter int unsigned DATA_WIDTH_SMALL = 16;
  parameter int unsigned NUM_REGS_LARGE = 32;
  parameter int unsigned DATA_WIDTH_LARGE = 64;
  
  // Clock and reset
  logic clk = 0;
  logic rst_n = 0;
  
  // Test signals for small register bank
  logic        we_small;
  logic [2:0]  addr_small;   // 3 bits for 8 registers
  logic [15:0] wdata_small;
  logic [15:0] rdata_small;
  logic        valid_small;
  
  // Test signals for large register bank
  logic        we_large;
  logic [4:0]  addr_large;   // 5 bits for 32 registers
  logic [63:0] wdata_large;
  logic [63:0] rdata_large;
  logic        valid_large;
  
  // Clock generation
  always #5 clk = ~clk;
  
  // Instantiate small register bank (8 registers, 16-bit data)
  generic_register_bank #(
    .NUM_REGISTERS(NUM_REGS_SMALL),
    .DATA_WIDTH(DATA_WIDTH_SMALL)
  ) small_register_bank (
    .clk(clk),
    .rst_n(rst_n),
    .write_enable(we_small),
    .address(addr_small),
    .write_data(wdata_small),
    .read_data(rdata_small),
    .valid_address(valid_small)
  );
  
  // Instantiate large register bank (32 registers, 64-bit data)
  generic_register_bank #(
    .NUM_REGISTERS(NUM_REGS_LARGE),
    .DATA_WIDTH(DATA_WIDTH_LARGE)
  ) large_register_bank (
    .clk(clk),
    .rst_n(rst_n),
    .write_enable(we_large),
    .address(addr_large),
    .write_data(wdata_large),
    .read_data(rdata_large),
    .valid_address(valid_large)
  );
  
  // Test class instances
  register_bank_config small_config;
  register_bank_config large_config;
  register_transaction trans;
  
  // Test procedure
  initial begin
    // Setup VCD dumping
    $dumpfile("register_bank_test_module.vcd");
    $dumpvars(0, register_bank_test_module);
    
    // Initialize configurations
    small_config = new(NUM_REGS_SMALL, DATA_WIDTH_SMALL);
    large_config = new(NUM_REGS_LARGE, DATA_WIDTH_LARGE);
    trans = new();
    
    $display("=== Generic Register Bank Test ===");
    $display();
    
    // Display configurations
    small_config.display_config();
    $display();
    large_config.display_config();
    $display();
    
    // Initialize signals
    we_small = 0;
    addr_small = 0;
    wdata_small = 0;
    we_large = 0;
    addr_large = 0;
    wdata_large = 0;
    
    // Reset sequence
    $display("Applying reset...");
    rst_n = 0;
    #20;
    rst_n = 1;
    #10;
    
    // Test small register bank
    $display("=== Testing Small Register Bank ===");
    test_register_bank_small();
    
    #50;
    
    // Test large register bank
    $display("=== Testing Large Register Bank ===");
    test_register_bank_large();
    
    #50;
    
    // Test address validation
    $display("=== Testing Address Validation ===");
    test_address_validation();
    
    #50;
    $display("=== Test Complete ===");
    $finish;
  end
  
  // Test task for small register bank
  task test_register_bank_small();
    $display("Writing test patterns to small register bank...");
    
    // Write to all registers
    for (int i = 0; i < NUM_REGS_SMALL; i++) begin
      @(posedge clk);
      we_small = 1;
      addr_small = i[2:0];  // Explicit width conversion
      wdata_small = 16'hA000 + 16'(i);  // Explicit width conversion
      @(posedge clk);
      we_small = 0;
      $display("  Written 0x%04X to register %0d", wdata_small, i);
    end
    
    $display("Reading back from small register bank...");
    
    // Read from all registers
    for (int i = 0; i < NUM_REGS_SMALL; i++) begin
      @(posedge clk);
      addr_small = i[2:0];  // Explicit width conversion
      @(posedge clk);
      #1; // Small delay for combinatorial read
      $display("  Register %0d: 0x%04X", i, rdata_small);
    end
  endtask
  
  // Test task for large register bank
  task test_register_bank_large();
    $display("Writing test patterns to large register bank...");
    
    // Write to first 8 registers
    for (int i = 0; i < 8; i++) begin
      @(posedge clk);
      we_large = 1;
      addr_large = i[4:0];  // Explicit width conversion
      wdata_large = 64'hDEADBEEF00000000 + 64'(i);  // Explicit width conversion
      @(posedge clk);
      we_large = 0;
      $display("  Written 0x%016X to register %0d", wdata_large, i);
    end
    
    $display("Reading back from large register bank...");
    
    // Read from first 8 registers
    for (int i = 0; i < 8; i++) begin
      @(posedge clk);
      addr_large = i[4:0];  // Explicit width conversion
      @(posedge clk);
      #1; // Small delay for combinatorial read
      $display("  Register %0d: 0x%016X", i, rdata_large);
    end
  endtask
  
  // Test address validation
  task test_address_validation();
    $display("Testing invalid addresses...");
    
    // Test invalid address on small bank
    @(posedge clk);
    addr_small = 3'h7; // Address 7 (max valid for 8-register bank)
    @(posedge clk);
    #1;
    $display("  Small bank addr=7, valid=%b, data=0x%04X", 
             valid_small, rdata_small);
    
    // Test valid address on large bank
    @(posedge clk);
    addr_large = 5'h1F; // Address 31 (valid for 32-register bank)
    @(posedge clk);
    #1;
    $display("  Large bank addr=31, valid=%b, data=0x%016X", 
             valid_large, rdata_large);
    
    // Test maximum address on large bank
    @(posedge clk);
    addr_large = 5'h1F; // Address 31 (maximum valid address)
    @(posedge clk);
    #1;
    $display("  Large bank max addr=31, valid=%b, data=0x%016X", 
             valid_large, rdata_large);
  endtask

endmodule
```

```systemverilog
// register_bank_pkg.sv
package register_bank_pkg;

  // Register bank configuration class
  class register_bank_config;
    int unsigned num_registers;
    int unsigned data_width;
    int unsigned addr_width;
    
    function new(int unsigned num_regs = 16, int unsigned width = 32);
      this.num_registers = num_regs;
      this.data_width = width;
      this.addr_width = $clog2(num_regs);
    endfunction
    
    function void display_config();
      $display("Register Bank Configuration:");
      $display("  Number of registers: %0d", num_registers);
      $display("  Data width: %0d bits", data_width);
      $display("  Address width: %0d bits", addr_width);
    endfunction
  endclass

  // Register bank transaction class
  class register_transaction;
    rand bit write_enable;
    rand bit [15:0] address;    // Maximum 16-bit address
    rand bit [31:0] write_data; // Maximum 32-bit data
    bit [31:0] read_data;
    
    constraint valid_operation {
      address inside {[0:15]};  // Default constraint, override in testbench
    }
    
    function void display_transaction();
      if (write_enable) begin
        $display("WRITE: addr=0x%02X, data=0x%08X", address, write_data);
      end else begin
        $display("READ:  addr=0x%02X, data=0x%08X", address, read_data);
      end
    endfunction
  endclass

endpackage
```

Verilator Simulation Output:
- Verilator: Walltime 79.952 s (elab=0.002, cvt=0.087, bld=79.434); cpu 0.075 s
on 1 threads; alloced 20.180 MB
Generic Register Bank instantiated:
  NUM_REGISTERS = 8
  DATA_WIDTH = 16
  ADDR_WIDTH = 3
Generic Register Bank instantiated:
  NUM_REGISTERS = 32
  DATA_WIDTH = 64
  ADDR_WIDTH = 5
=== Generic Register Bank Test ===

Register Bank Configuration:
  Number of registers: 8
  Data width: 16 bits
  Address width: 3 bits

Register Bank Configuration:
  Number of registers: 32
  Data width: 64 bits
  Address width: 5 bits

Applying reset...
=== Testing Small Register Bank ===
Writing test patterns to small register bank...
  Written 0xa000 to register 0
  Written 0xa001 to register 1
  Written 0xa002 to register 2
  Written 0xa003 to register 3
  Written 0xa004 to register 4
  Written 0xa005 to register 5
  Written 0xa006 to register 6
  Written 0xa007 to register 7
Reading back from small register bank...
  Register 0: 0xa000
  Register 1: 0xa001
  Re

0

### 5. Parameterized Memory Model
**Description:** Simple memory model with configurable address width and data width. Basic memory read/write operations.

In [8]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_5__parameterized_memory_model/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// parameterized_memory_model.sv
// Simple parameterized memory model with configurable width

package memory_model_pkg;

  // Parameterized memory model class
  class ParameterizedMemory #(
    parameter int ADDR_WIDTH = 8,
    parameter int DATA_WIDTH = 32
  );
    
    // Memory array - associative array for sparse memory
    local bit [DATA_WIDTH-1:0] memory_array[bit [ADDR_WIDTH-1:0]];
    
    // Statistics
    local int read_count;
    local int write_count;
    
    // Constructor
    function new();
      read_count = 0;
      write_count = 0;
      $display("Memory model created - ADDR_WIDTH: %0d, DATA_WIDTH: %0d", 
               ADDR_WIDTH, DATA_WIDTH);
    endfunction
    
    // Write operation
    function void write(bit [ADDR_WIDTH-1:0] addr, 
                       bit [DATA_WIDTH-1:0] data);
      memory_array[addr] = data;
      write_count++;
      $display("WRITE: addr=0x%0h, data=0x%0h", addr, data);
    endfunction
    
    // Read operation
    function bit [DATA_WIDTH-1:0] read(bit [ADDR_WIDTH-1:0] addr);
      bit [DATA_WIDTH-1:0] data;
      
      /* verilator lint_off WIDTHTRUNC */
      if (memory_array.exists(addr)) begin
      /* verilator lint_on WIDTHTRUNC */
        data = memory_array[addr];
      end else begin
        data = {DATA_WIDTH{1'bx}};  // Return X for uninitialized
        $display("WARNING: Reading uninitialized address 0x%0h", addr);
      end
      
      read_count++;
      $display("READ:  addr=0x%0h, data=0x%0h", addr, data);
      return data;
    endfunction
    
    // Clear memory
    function void clear();
      memory_array.delete();
      $display("Memory cleared");
    endfunction
    
    // Get statistics
    function void print_stats();
      $display("Memory Statistics:");
      $display("  Read operations:  %0d", read_count);
      $display("  Write operations: %0d", write_count);
      $display("  Memory entries:   %0d", memory_array.size());
    endfunction
    
    // Memory dump (for debugging)
    function void dump_memory();
      bit [ADDR_WIDTH-1:0] addr;
      
      $display("Memory Dump:");
      if (memory_array.size() == 0) begin
        $display("  Memory is empty");
      end else begin
        /* verilator lint_off WIDTHTRUNC */
        if (memory_array.first(addr)) begin
          do begin
            $display("  [0x%0h] = 0x%0h", addr, memory_array[addr]);
          end while (memory_array.next(addr));
        end
        /* verilator lint_on WIDTHTRUNC */
      end
    endfunction
    
  endclass

endpackage

// Simple module wrapper for Verilator compatibility
module parameterized_memory_model;
  import memory_model_pkg::*;
  
  initial begin
    $display("Parameterized Memory Model - Design Module");
    $display("This module contains the memory model package");
  end
  
endmodule
```

```systemverilog
// parameterized_memory_model_testbench.sv
// Testbench for parameterized memory model

module memory_model_test_bench;
  import memory_model_pkg::*;
  
  // Instantiate design under test
  parameterized_memory_model MEMORY_MODEL_INSTANCE();
  
  // Test variables
  typedef ParameterizedMemory #(.ADDR_WIDTH(8), .DATA_WIDTH(32)) Memory32;
  typedef ParameterizedMemory #(.ADDR_WIDTH(4), .DATA_WIDTH(16)) Memory16;
  
  Memory32 mem32;
  Memory16 mem16;
  
  bit [31:0] read_data_32;
  bit [15:0] read_data_16;
  
  initial begin
    // Setup waveform dumping
    $dumpfile("memory_model_test_bench.vcd");
    $dumpvars(0, memory_model_test_bench);
    
    $display("=== Parameterized Memory Model Test ===");
    $display();
    
    // Test 32-bit memory
    $display("--- Testing 32-bit Memory (8-bit addr) ---");
    mem32 = new();
    
    // Write some data
    mem32.write(8'h00, 32'hDEADBEEF);
    mem32.write(8'h01, 32'hCAFEBABE);
    mem32.write(8'hFF, 32'h12345678);
    
    $display();
    
    // Read back data
    read_data_32 = mem32.read(8'h00);
    read_data_32 = mem32.read(8'h01);
    read_data_32 = mem32.read(8'hFF);
    
    // Try reading uninitialized address
    $display();
    read_data_32 = mem32.read(8'h10);
    
    $display();
    mem32.dump_memory();
    
    $display();
    mem32.print_stats();
    
    $display();
    $display("--- Testing 16-bit Memory (4-bit addr) ---");
    mem16 = new();
    
    // Write some data
    mem16.write(4'h0, 16'hABCD);
    mem16.write(4'h1, 16'h1234);
    mem16.write(4'hF, 16'h9876);
    
    $display();
    
    // Read back data
    read_data_16 = mem16.read(4'h0);
    read_data_16 = mem16.read(4'h1);
    read_data_16 = mem16.read(4'hF);
    
    $display();
    mem16.dump_memory();
    
    $display();
    mem16.print_stats();
    
    // Test clear functionality
    $display();
    $display("--- Testing Clear Functionality ---");
    mem32.clear();
    mem32.dump_memory();
    mem32.print_stats();
    
    $display();
    $display("Test completed successfully!");
    
    // Small delay before finishing
    #10;
    
  end
  
endmodule
```

Verilator Simulation Output:
Parameterized Memory Model - Design Module
This module contains the memory model package
=== Parameterized Memory Model Test ===

--- Testing 32-bit Memory (8-bit addr) ---
Memory model created - ADDR_WIDTH: 8, DATA_WIDTH: 32
WRITE: addr=0x0, data=0xdeadbeef
WRITE: addr=0x1, data=0xcafebabe
WRITE: addr=0xff, data=0x12345678

READ:  addr=0x0, data=0xdeadbeef
READ:  addr=0x1, data=0xcafebabe
READ:  addr=0xff, data=0x12345678

READ:  addr=0x10, data=0x0

Memory Dump:
  [0x0] = 0xdeadbeef
  [0x1] = 0xcafebabe
  [0xff] = 0x12345678

Memory Statistics:
  Read operations:  4
  Write operations: 3
  Memory entries:   3

--- Testing 16-bit Memory (4-bit addr) ---
Memory model created - ADDR_WIDTH: 4, DATA_WIDTH: 16
WRITE: addr=0x0, data=0xabcd
WRITE: addr=0x1, data=0x1234
WRITE: addr=0xf, data=0x9876

READ:  addr=0x0, data=0xabcd
READ:  addr=0x1, data=0x1234
READ:  addr=0xf, data=0x9876

Memory Dump:
  [0x0] = 0xabcd
  [0x1] = 0x1234
  [0xf] = 0x9876

Memory Statist

0

## Advanced Parameterized Classes with Type Parameters

### 6. Generic Array Container
**Description:** Container class that can hold any data type (int, string, custom structs). Shows add(), get(), and display() methods.

In [4]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_6__generic_array_container/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// generic_array_container.sv
package generic_container_pkg;

  // Custom struct type for testing
  typedef struct {
    string name;
    int    age;
  } person_t;

  // Generic array container class
  class generic_array_container #(type T = int);
    
    // Private data members
    local T data_array[];
    local int size;
    local int capacity;
    
    // Constructor
    function new(int initial_capacity = 10);
      capacity = initial_capacity;
      size = 0;
      data_array = new[capacity];
    endfunction
    
    // Add element to container
    function void add(T item);
      if (size >= capacity) begin
        // Double capacity when full
        T temp_array[] = new[capacity * 2];
        for (int i = 0; i < size; i++) begin
          temp_array[i] = data_array[i];
        end
        data_array = temp_array;
        capacity *= 2;
        $display("Container resized to capacity: %0d", capacity);
      end
      data_array[size] = item;
      size++;
    endfunction
    
    // Get element from container
    function T get(int index);
      if (index >= 0 && index < size) begin
        return data_array[index];
      end else begin
        $error("Index %0d out of bounds (size=%0d)", index, size);
        return data_array[0]; // Return first element as default
      end
    endfunction
    
    // Display all elements
    function void display();
      $display("Container contents (size=%0d, capacity=%0d):", 
               size, capacity);
      for (int i = 0; i < size; i++) begin
        $display("  [%0d]: %p", i, data_array[i]);
      end
    endfunction
    
    // Get current size
    function int get_size();
      return size;
    endfunction
    
    // Get current capacity
    function int get_capacity();
      return capacity;
    endfunction
    
    // Check if container is empty
    function bit is_empty();
      return (size == 0);
    endfunction
    
    // Clear all elements
    function void clear();
      size = 0;
    endfunction
    
  endclass

endpackage
```

```systemverilog
// generic_array_container_testbench.sv
module generic_array_container_testbench;
  
  import generic_container_pkg::*;
  
  // Test containers for different data types
  generic_array_container #(int) int_container;
  generic_array_container #(string) string_container;
  generic_array_container #(person_t) person_container;
  
  // Variables for testing
  person_t alice, bob, charlie, retrieved_person;
  int dummy;
  
  initial begin
    // Dump waves
    $dumpfile("generic_array_container_testbench.vcd");
    $dumpvars(0, generic_array_container_testbench);
    
    $display("=== Generic Array Container Test ===");
    $display();
    
    // Test 1: Integer container
    $display("--- Test 1: Integer Container ---");
    int_container = new(3); // Small initial capacity for testing resize
    
    // Add integers
    for (int i = 1; i <= 5; i++) begin
      int_container.add(i * 10);
      $display("Added: %0d", i * 10);
    end
    
    int_container.display();
    
    // Test get method
    $display("Getting element at index 2: %0d", int_container.get(2));
    $display("Container size: %0d", int_container.get_size());
    $display();
    
    // Test 2: String container
    $display("--- Test 2: String Container ---");
    string_container = new(2);
    
    string_container.add("Hello");
    string_container.add("World");
    string_container.add("SystemVerilog");
    string_container.add("Generic");
    string_container.add("Container");
    
    string_container.display();
    $display("First string: %s", string_container.get(0));
    $display("Last string: %s", 
             string_container.get(string_container.get_size()-1));
    $display();
    
    // Test 3: Custom struct container
    $display("--- Test 3: Person Struct Container ---");
    person_container = new(2);
    
    alice = '{"Alice", 25};
    bob = '{"Bob", 30};
    charlie = '{"Charlie", 35};
    
    person_container.add(alice);
    person_container.add(bob);
    person_container.add(charlie);
    
    person_container.display();
    
    retrieved_person = person_container.get(1);
    $display("Retrieved person: name=%s, age=%0d", 
             retrieved_person.name, retrieved_person.age);
    $display();
    
    // Test 4: Error handling
    $display("--- Test 4: Error Handling ---");
    $display("Attempting to access invalid index...");
    dummy = int_container.get(100); // Should generate error
    $display();
    
    // Test 5: Container operations
    $display("--- Test 5: Container Operations ---");
    $display("String container empty? %s", 
             string_container.is_empty() ? "Yes" : "No");
    
    string_container.clear();
    $display("After clear, string container empty? %s", 
             string_container.is_empty() ? "Yes" : "No");
    $display("String container size: %0d", string_container.get_size());
    
    $display();
    $display("=== Test Complete ===");
    
    #1;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Generic Array Container Test ===

--- Test 1: Integer Container ---
Added: 10
Added: 20
Added: 30
Container resized to capacity: 6
Added: 40
Added: 50
Container contents (size=5, capacity=6):
  [0]:         10
  [1]:         20
  [2]:         30
  [3]:         40
  [4]:         50
Getting element at index 2: 30
Container size: 5

--- Test 2: String Container ---
Container resized to capacity: 4
Container resized to capacity: 8
Container contents (size=5, capacity=8):
  [0]: "Hello"
  [1]: "World"
  [2]: "SystemVerilog"
  [3]: "Generic"
  [4]: "Container"
First string: Hello
Last string: Container

--- Test 3: Person Struct Container ---
Container resized to capacity: 4
Container contents (size=3, capacity=4):
  [0]: '{name:"Alice", age:'h19}
  [1]: '{name:"Bob", age:'h1e}
  [2]: '{name:"Charlie", age:'h23}
Retrieved person: name=Bob, age=30

--- Test 4: Error Handling ---
Attempting to access invalid index...
Process finished with return code: 139

Dock

139

### 7. Type-Safe Queue
**Description:** Queue implementation that works with different data types. Demonstrates enqueue/dequeue operations with type safety.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_7__type_safe_queue/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// type_safe_queue_design.sv
// Type-safe queue implementation with parameterized data types

package type_safe_queue_pkg;

  // Base queue interface for type safety
  virtual class BaseQueue;
    pure virtual function int size();
    pure virtual function bit is_empty();
    pure virtual function bit is_full();
    pure virtual function void clear();
  endclass

  // Parameterized type-safe queue class
  class TypeSafeQueue #(type T = int, int MAX_SIZE = 16) extends BaseQueue;
    
    // Internal storage array
    protected T queue_data[MAX_SIZE];
    protected int head_ptr;
    protected int tail_ptr;
    protected int count;
    
    // Constructor
    function new();
      head_ptr = 0;
      tail_ptr = 0;
      count = 0;
    endfunction
    
    // Enqueue operation - add element to rear
    virtual function bit enqueue(T item);
      if (is_full()) begin
        $display("ERROR: Queue is full, cannot enqueue item");
        return 0;
      end
      
      queue_data[tail_ptr] = item;
      tail_ptr = (tail_ptr + 1) % MAX_SIZE;
      count++;
      
      $display("Enqueued item (Queue size: %0d)", count);
      return 1;
    endfunction
    
    // Dequeue operation - remove element from front
    virtual function bit dequeue(output T item);
      if (is_empty()) begin
        $display("ERROR: Queue is empty, cannot dequeue item");
        return 0;
      end
      
      item = queue_data[head_ptr];
      head_ptr = (head_ptr + 1) % MAX_SIZE;
      count--;
      
      $display("Dequeued item (Queue size: %0d)", count);
      return 1;
    endfunction
    
    // Peek at front element without removing it
    virtual function bit peek_front(output T item);
      if (is_empty()) begin
        $display("ERROR: Queue is empty, cannot peek");
        return 0;
      end
      
      item = queue_data[head_ptr];
      $display("Peek front operation completed");
      return 1;
    endfunction
    
    // Peek at rear element without removing it
    virtual function bit peek_rear(output T item);
      int rear_idx;
      
      if (is_empty()) begin
        $display("ERROR: Queue is empty, cannot peek");
        return 0;
      end
      
      rear_idx = (tail_ptr - 1 + MAX_SIZE) % MAX_SIZE;
      item = queue_data[rear_idx];
      $display("Peek rear operation completed");
      return 1;
    endfunction
    
    // Get current queue size
    virtual function int size();
      return count;
    endfunction
    
    // Check if queue is empty
    virtual function bit is_empty();
      return (count == 0);
    endfunction
    
    // Check if queue is full
    virtual function bit is_full();
      return (count == MAX_SIZE);
    endfunction
    
    // Clear all elements
    virtual function void clear();
      head_ptr = 0;
      tail_ptr = 0;
      count = 0;
      $display("Queue cleared");
    endfunction
    
    // Display queue contents
    virtual function void display_queue();
      int i, idx;
      
      $display("=== Queue Contents ===");
      $display("Size: %0d/%0d", count, MAX_SIZE);
      $display("Head: %0d, Tail: %0d", head_ptr, tail_ptr);
      
      if (is_empty()) begin
        $display("Queue is empty");
      end else begin
        $display("Queue has %0d elements", count);
      end
      $display("===================");
    endfunction
    
  endclass

  // Specialized queue for string data
  class StringQueue extends TypeSafeQueue#(string, 8);
    
    // Override enqueue for string-specific messaging
    virtual function bit enqueue(string item);
      if (is_full()) begin
        $display("ERROR: String queue is full, cannot enqueue item");
        return 0;
      end
      
      queue_data[tail_ptr] = item;
      tail_ptr = (tail_ptr + 1) % MAX_SIZE;
      count++;
      
      $display("Enqueued string: \"%s\" (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override dequeue for string-specific messaging
    virtual function bit dequeue(output string item);
      if (is_empty()) begin
        $display("ERROR: String queue is empty, cannot dequeue item");
        return 0;
      end
      
      item = queue_data[head_ptr];
      head_ptr = (head_ptr + 1) % MAX_SIZE;
      count--;
      
      $display("Dequeued string: \"%s\" (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override display for string-specific formatting
    virtual function void display_queue();
      int i, idx;
      
      $display("=== String Queue Contents ===");
      $display("Size: %0d/%0d", count, MAX_SIZE);
      
      if (is_empty()) begin
        $display("Queue is empty");
      end else begin
        $display("Elements:");
        for (i = 0; i < count; i++) begin
          idx = (head_ptr + i) % MAX_SIZE;
          $display("  [%0d]: \"%s\"", i, queue_data[idx]);
        end
      end
      $display("============================");
    endfunction
    
  endclass

  // Specialized queue for integer data
  class IntQueue extends TypeSafeQueue#(int, 10);
    
    // Override enqueue for int-specific messaging
    virtual function bit enqueue(int item);
      if (is_full()) begin
        $display("ERROR: Int queue is full, cannot enqueue item");
        return 0;
      end
      
      queue_data[tail_ptr] = item;
      tail_ptr = (tail_ptr + 1) % MAX_SIZE;
      count++;
      
      $display("Enqueued int: %0d (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override dequeue for int-specific messaging
    virtual function bit dequeue(output int item);
      if (is_empty()) begin
        $display("ERROR: Int queue is empty, cannot dequeue item");
        return 0;
      end
      
      item = queue_data[head_ptr];
      head_ptr = (head_ptr + 1) % MAX_SIZE;
      count--;
      
      $display("Dequeued int: %0d (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override display for int-specific formatting
    virtual function void display_queue();
      int i, idx;
      
      $display("=== Int Queue Contents ===");
      $display("Size: %0d/%0d", count, MAX_SIZE);
      
      if (is_empty()) begin
        $display("Queue is empty");
      end else begin
        $write("Elements: ");
        for (i = 0; i < count; i++) begin
          idx = (head_ptr + i) % MAX_SIZE;
          $write("%0d ", queue_data[idx]);
        end
        $display();
      end
      $display("=========================");
    endfunction
    
  endclass

  // Specialized queue for byte data
  class ByteQueue extends TypeSafeQueue#(bit [7:0], 6);
    
    // Override enqueue for byte-specific messaging
    virtual function bit enqueue(bit [7:0] item);
      if (is_full()) begin
        $display("ERROR: Byte queue is full, cannot enqueue item");
        return 0;
      end
      
      queue_data[tail_ptr] = item;
      tail_ptr = (tail_ptr + 1) % MAX_SIZE;
      count++;
      
      $display("Enqueued byte: 0x%02h (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override dequeue for byte-specific messaging
    virtual function bit dequeue(output bit [7:0] item);
      if (is_empty()) begin
        $display("ERROR: Byte queue is empty, cannot dequeue item");
        return 0;
      end
      
      item = queue_data[head_ptr];
      head_ptr = (head_ptr + 1) % MAX_SIZE;
      count--;
      
      $display("Dequeued byte: 0x%02h (Queue size: %0d)", item, count);
      return 1;
    endfunction
    
    // Override display for byte-specific formatting
    virtual function void display_queue();
      int i, idx;
      
      $display("=== Byte Queue Contents ===");
      $display("Size: %0d/%0d", count, MAX_SIZE);
      
      if (is_empty()) begin
        $display("Queue is empty");
      end else begin
        $write("Elements: ");
        for (i = 0; i < count; i++) begin
          idx = (head_ptr + i) % MAX_SIZE;
          $write("0x%02h ", queue_data[idx]);
        end
        $display();
      end
      $display("=========================");
    endfunction
    
  endclass

  // Queue manager for handling multiple queue types
  class QueueManager;
    
    // Different queue instances
    IntQueue int_queue;
    ByteQueue byte_queue;
    StringQueue string_queue;
    
    function new();
      int_queue = new();
      byte_queue = new();
      string_queue = new();
    endfunction
    
    // Demonstrate operations on different queue types
    function void demonstrate_operations();
      bit success;
      
      $display("\n=== Queue Manager Demo ===");
      
      // Test integer queue
      $display("\n--- Integer Queue Test ---");
      success = int_queue.enqueue(42);
      if (!success) $display("Failed to enqueue 42");
      
      success = int_queue.enqueue(100);
      if (!success) $display("Failed to enqueue 100");
      
      success = int_queue.enqueue(255);
      if (!success) $display("Failed to enqueue 255");
      
      int_queue.display_queue();
      
      // Test byte queue
      $display("\n--- Byte Queue Test ---");
      success = byte_queue.enqueue(8'hFF);
      if (!success) $display("Failed to enqueue 0xFF");
      
      success = byte_queue.enqueue(8'hAA);
      if (!success) $display("Failed to enqueue 0xAA");
      
      success = byte_queue.enqueue(8'h55);
      if (!success) $display("Failed to enqueue 0x55");
      
      byte_queue.display_queue();
      
      // Test string queue
      $display("\n--- String Queue Test ---");
      success = string_queue.enqueue("Hello");
      if (!success) $display("Failed to enqueue Hello");
      
      success = string_queue.enqueue("World");
      if (!success) $display("Failed to enqueue World");
      
      success = string_queue.enqueue("SystemVerilog");
      if (!success) $display("Failed to enqueue SystemVerilog");
      
      string_queue.display_queue();
      
      $display("\n=========================");
    endfunction
    
  endclass

endpackage

// Top-level design module
module type_safe_queue_design();
  
  import type_safe_queue_pkg::*;
  
  // Queue instances for testing
  QueueManager queue_mgr;
  
  initial begin
    $display("=== Type-Safe Queue Design Demo ===");
    
    // Create queue manager
    queue_mgr = new();
    
    // Run demonstration
    queue_mgr.demonstrate_operations();
    
    $display("\n=== Design Demo Complete ===");
  end

endmodule
```

```systemverilog
// type_safe_queue_testbench.sv
// Comprehensive testbench for type-safe queue implementation

module type_safe_queue_testbench;

  import type_safe_queue_pkg::*;
  
  // Test queue instances
  IntQueue small_int_queue;
  ByteQueue word_queue;
  StringQueue test_string_queue;
  
  // Test data
  int test_values[] = {10, 20, 30, 40, 50, 60, 70};
  bit [7:0] test_bytes[] = {8'hAB, 8'h12, 8'hFE, 8'h56};
  string test_strings[] = {"First", "Second", "Third", "Fourth", "Fifth"};
  
  // Test results tracking
  int test_pass_count = 0;
  int test_fail_count = 0;
  
  // Test assertion macro
  `define ASSERT(condition, message) \
    if (condition) begin \
      $display("PASS: %s", message); \
      test_pass_count++; \
    end else begin \
      $display("FAIL: %s", message); \
      test_fail_count++; \
    end
  
  // Instantiate design under test
  type_safe_queue_design DESIGN_INSTANCE();
  
  initial begin
    // Setup waveform dumping
    $dumpfile("type_safe_queue_testbench.vcd");
    $dumpvars(0, type_safe_queue_testbench);
    
    $display("=== Type-Safe Queue Testbench Started ===");
    
    // Initialize queue instances
    small_int_queue = new();
    word_queue = new();
    test_string_queue = new();
    
    // Run all tests
    test_basic_operations();
    test_boundary_conditions();
    test_type_safety();
    test_queue_overflow_underflow();
    test_peek_operations();
    test_mixed_operations();
    
    // Display final results
    display_test_results();
    
    #10; // Wait before finishing
    $display("=== Testbench Complete ===");
    $finish;
  end
  
  // Test basic enqueue/dequeue operations
  task automatic test_basic_operations();
    int dequeued_value;
    bit success;
    
    $display("\n=== Test: Basic Operations ===");
    
    // Test enqueue
    success = small_int_queue.enqueue(100);
    `ASSERT(success, "Enqueue operation successful");
    `ASSERT(small_int_queue.size() == 1, "Queue size is 1 after enqueue");
    
    // Test dequeue
    success = small_int_queue.dequeue(dequeued_value);
    `ASSERT(success, "Dequeue operation successful");
    `ASSERT(dequeued_value == 100, "Dequeued correct value");
    `ASSERT(small_int_queue.size() == 0, "Queue size is 0 after dequeue");
    
    small_int_queue.display_queue();
  endtask
  
  // Test boundary conditions
  task automatic test_boundary_conditions();
    int dummy_value;
    bit success;
    int i;
    
    $display("\n=== Test: Boundary Conditions ===");
    
    // Test empty queue
    `ASSERT(small_int_queue.is_empty(), "New queue is empty");
    `ASSERT(!small_int_queue.is_full(), "New queue is not full");
    
    // Fill queue to capacity
    for (i = 0; i < 10; i++) begin
      success = small_int_queue.enqueue(i * 10);
      `ASSERT(success, $sformatf("Enqueue item %0d successful", i));
    end
    
    // Test full queue
    `ASSERT(small_int_queue.is_full(), "Queue is full after filling");
    `ASSERT(small_int_queue.size() == 10, "Queue size is 10 when full");
    
    small_int_queue.display_queue();
    
    // Clear queue for next test
    small_int_queue.clear();
    `ASSERT(small_int_queue.is_empty(), "Queue is empty after clear");
  endtask
  
  // Test type safety with different data types
  task automatic test_type_safety();
    bit [7:0] byte_value;
    string string_value;
    bit success;
    
    $display("\n=== Test: Type Safety ===");
    
    // Test byte queue
    success = word_queue.enqueue(8'hDE);
    `ASSERT(success, "Byte enqueue successful");
    
    success = word_queue.enqueue(8'hAD);
    `ASSERT(success, "Second byte enqueue successful");
    
    word_queue.display_queue();
    
    success = word_queue.dequeue(byte_value);
    `ASSERT(success && byte_value == 8'hDE, "Byte dequeue correct");
    
    // Test string queue
    success = test_string_queue.enqueue("TypeSafe");
    `ASSERT(success, "String enqueue successful");
    
    success = test_string_queue.enqueue("Queue");
    `ASSERT(success, "Second string enqueue successful");
    
    test_string_queue.display_queue();
    
    success = test_string_queue.dequeue(string_value);
    `ASSERT(success && string_value == "TypeSafe", "String dequeue correct");
  endtask
  
  // Test overflow and underflow conditions
  task automatic test_queue_overflow_underflow();
    int dummy_value;
    bit success;
    int i;
    
    $display("\n=== Test: Overflow/Underflow ===");
    
    // Test underflow on empty queue
    success = small_int_queue.dequeue(dummy_value);
    `ASSERT(!success, "Dequeue from empty queue fails correctly");
    
    // Fill queue beyond capacity
    for (i = 0; i < 12; i++) begin
      success = small_int_queue.enqueue(i);
      if (i < 10) begin
        `ASSERT(success, $sformatf("Enqueue %0d successful", i));
      end else begin
        `ASSERT(!success, $sformatf("Enqueue %0d fails when full", i));
      end
    end
    
    small_int_queue.display_queue();
    small_int_queue.clear();
  endtask
  
  // Test peek operations
  task automatic test_peek_operations();
    int front_value, rear_value;
    bit success;
    
    $display("\n=== Test: Peek Operations ===");
    
    // Test peek on empty queue
    success = small_int_queue.peek_front(front_value);
    `ASSERT(!success, "Peek front on empty queue fails correctly");
    
    // Add some elements
    success = small_int_queue.enqueue(111);
    `ASSERT(success, "Enqueue 111 successful");
    
    success = small_int_queue.enqueue(222);
    `ASSERT(success, "Enqueue 222 successful");
    
    success = small_int_queue.enqueue(333);
    `ASSERT(success, "Enqueue 333 successful");
    
    // Test peek operations
    success = small_int_queue.peek_front(front_value);
    `ASSERT(success && front_value == 111, "Peek front returns correct value");
    
    success = small_int_queue.peek_rear(rear_value);
    `ASSERT(success && rear_value == 333, "Peek rear returns correct value");
    
    // Verify peek doesn't modify queue
    `ASSERT(small_int_queue.size() == 3, "Queue size unchanged after peek");
    
    small_int_queue.display_queue();
    small_int_queue.clear();
  endtask
  
  // Test mixed operations scenario
  task automatic test_mixed_operations();
    int values_to_test[] = {5, 15, 25, 35, 45};
    int dequeued_val;
    bit success;
    int i;
    
    $display("\n=== Test: Mixed Operations ===");
    
    // Enqueue some values
    for (i = 0; i < 3; i++) begin
      success = small_int_queue.enqueue(values_to_test[i]);
      `ASSERT(success, $sformatf("Mixed test enqueue %0d", i));
    end
    
    // Dequeue one
    success = small_int_queue.dequeue(dequeued_val);
    `ASSERT(success && dequeued_val == 5, "Mixed test dequeue correct");
    
    // Enqueue more
    for (i = 3; i < 5; i++) begin
      success = small_int_queue.enqueue(values_to_test[i]);
      `ASSERT(success, $sformatf("Mixed test enqueue %0d", i));
    end
    
    small_int_queue.display_queue();
    
    // Verify final state
    `ASSERT(small_int_queue.size() == 4, "Mixed test final size correct");
    
    // Test circular buffer behavior
    for (i = 0; i < 6; i++) begin
      success = small_int_queue.enqueue(50 + i);
      `ASSERT(success, $sformatf("Circular buffer enqueue %0d successful", i));
    end
    `ASSERT(small_int_queue.is_full(), "Queue is full after circular fill");
    
    small_int_queue.display_queue();
    small_int_queue.clear();
  endtask
  
  // Display final test results
  task automatic display_test_results();
    $display("\n=== Final Test Results ===");
    $display("Tests Passed: %0d", test_pass_count);
    $display("Tests Failed: %0d", test_fail_count);
    $display("Total Tests:  %0d", test_pass_count + test_fail_count);
    
    if (test_fail_count == 0) begin
      $display("*** ALL TESTS PASSED! ***");
    end else begin
      $display("*** SOME TESTS FAILED ***");
    end
    $display("=========================");
  endtask

endmodule
```

Verilator Simulation Output:
=== Type-Safe Queue Design Demo ===

=== Queue Manager Demo ===

--- Integer Queue Test ---
Enqueued int: 42 (Queue size: 1)
Enqueued int: 100 (Queue size: 2)
Enqueued int: 255 (Queue size: 3)
=== Int Queue Contents ===
Size: 3/10
Elements: 42 100 255

--- Byte Queue Test ---
Enqueued byte: 0xff (Queue size: 1)
Enqueued byte: 0xaa (Queue size: 2)
Enqueued byte: 0x55 (Queue size: 3)
=== Byte Queue Contents ===
Size: 3/6
Elements: 0xff 0xaa 0x55

--- String Queue Test ---
Enqueued string: "Hello" (Queue size: 1)
Enqueued string: "World" (Queue size: 2)
Enqueued string: "SystemVerilog" (Queue size: 3)
=== String Queue Contents ===
Size: 3/8
Elements:
  [0]: "Hello"
  [1]: "World"
  [2]: "SystemVerilog"


=== Design Demo Complete ===
=== Type-Safe Queue Testbench Started ===

=== Test: Basic Operations ===
Enqueued int: 100 (Queue size: 1)
PASS: Enqueue operation successful
PASS: Queue size is 1 after enqueue
Dequeued int: 100 (Queue size: 0)
PASS: Dequeue oper

0

### 8. Generic Pair Class
**Description:** Simple class that holds two values of potentially different types. Shows parameterization with multiple type parameters.

In [4]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_8__generic_pair/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// Design module: generic_pair
module generic_pair #(
    type T1,
    type T2
) (
    input  T1 value1,
    input  T2 value2,
    output T1 first,
    output T2 second
);
    assign first = value1;
    assign second = value2;
endmodule
```

```systemverilog
// Testbench module: generic_pair_tb
module generic_pair_tb;
    // Instantiate the design module
    generic_pair #(
        .T1(integer),
        .T2(string)
    ) dut (
        .value1(10),
        .value2("Hello"),
        .first(out_first),
        .second(out_second)
    );

    integer out_first;
    string out_second;

    initial begin
        $dumpfile("generic_pair_tb.vcd");
        $dumpvars(0, generic_pair_tb);

        $display("=== Generic Pair Testbench Started ===");

        $display("Testing generic pair...");
        $display("Applying inputs: value1 = 10, value2 = 'Hello'");

        #10; // Wait for 10 time units
        $display("Time %0t: Checking outputs...", $time);
        $display("Expected: first = 10, second = 'Hello'");
        $display("Got: first = %d, second = %s", out_first, out_second);

        if (out_first == 10 && out_second == "Hello") begin
            $display("Test generic_pair: PASSED");
        end else begin
            $display("Test generic_pair: FAILED");
        end

        $display("=== Testbench Completed ===");
        $finish;
    end
endmodule
```

Verilator Simulation Output:
=== Generic Pair Testbench Started ===
Testing generic pair...
Applying inputs: value1 = 10, value2 = 'Hello'
Time 10: Checking outputs...
Expected: first = 10, second = 'Hello'
Got: first =          10, second = Hello
Test generic_pair: PASSED
=== Testbench Completed ===
Process finished with return code: 0
Removing Chapter_11_examples/example_8__generic_pair/obj_dir directory...
Chapter_11_examples/example_8__generic_pair/obj_dir removed successfully.


0

### 9. Flexible Scoreboard
**Description:** Scoreboard class that can compare any data type. Demonstrates generic comparison and tracking functionality.

In [5]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_9__scoreboard/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
module scoreboard #(type T = int) (
    input  logic clk,
    input  T expected,
    input  T actual,
    input  logic check
);

    integer pass_count = 0;
    integer fail_count = 0;

    always @(posedge clk) begin
        if (check) begin
            if (expected == actual) begin
                pass_count++;
                $display("Test PASSED: Expected %d, Got %d", expected, actual);
            end else begin
                fail_count++;
                $display("Test FAILED: Expected %d, Got %d", expected, actual);
            end
        end
    end

    always @(posedge clk) begin
        $display("Scoreboard: %d Passed, %d Failed", pass_count, fail_count);
    end

endmodule
```

```systemverilog
module scoreboard_tb;

    logic clk;
    integer expected;
    integer actual;
    logic check;

    scoreboard #(int) scoreboard_inst (
        .clk(clk),
        .expected(expected),
        .actual(actual),
        .check(check)
    );

    initial begin
        $dumpfile("scoreboard_tb.vcd");
        $dumpvars(0, scoreboard_tb);

        $display("=== Scoreboard Testbench Started ===");

        clk = 0;
        expected = 1;
        actual = 1;
        check = 1;

        #10 clk = 1;
        #10 clk = 0;

        expected = 2;
        actual = 2;
        check = 1;

        #10 clk = 1;
        #10 clk = 0;

        expected = 3;
        actual = 4;
        check = 1;

        #10 clk = 1;
        #10 clk = 0;

        $display("=== Testbench Completed ===");
        $finish;
    end

endmodule
```

Verilator Simulation Output:
=== Scoreboard Testbench Started ===
Scoreboard:           0 Passed,           0 Failed
Test PASSED: Expected           1, Got           1
Scoreboard:           1 Passed,           0 Failed
Test PASSED: Expected           2, Got           2
Scoreboard:           2 Passed,           0 Failed
Test FAILED: Expected           3, Got           4
=== Testbench Completed ===
Process finished with return code: 0
Removing Chapter_11_examples/example_9__scoreboard/obj_dir directory...
Chapter_11_examples/example_9__scoreboard/obj_dir removed successfully.


0

### 10. Universal Cache
**Description:** Simple cache implementation that works with any key-value pair types. Basic get/put operations with different data types.

In [6]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_11_examples/example_10__universal_cache/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
module universal_cache #(
    parameter int unsigned KEY_WIDTH = 8,
    parameter int unsigned VALUE_WIDTH = 8,
    parameter int unsigned CACHE_SIZE = 4
) (
    input logic clk,
    input logic rst,
    input logic [KEY_WIDTH-1:0] key,
    input logic [VALUE_WIDTH-1:0] value,
    input logic put,
    output logic [VALUE_WIDTH-1:0] output_value,
    output logic hit
);

logic [CACHE_SIZE-1:0][KEY_WIDTH-1:0] cache_keys;
logic [CACHE_SIZE-1:0][VALUE_WIDTH-1:0] cache_values;

always_ff @(posedge clk or posedge rst) begin
    if (rst) begin
        cache_keys <= 0;
        cache_values <= 0;
    end else begin
        if (put) begin
            cache_keys[0] <= key;
            cache_values[0] <= value;
            for (int i = 1; i < CACHE_SIZE; i++) begin
                cache_keys[i] <= cache_keys[i-1];
                cache_values[i] <= cache_values[i-1];
            end
        end
    end
end

always_comb begin
    hit = 0;
    output_value = 0;
    for (int i = 0; i < CACHE_SIZE; i++) begin
        if (cache_keys[i] == key) begin
            hit = 1;
            output_value = cache_values[i];
            break;
        end
    end
end

endmodule
```

```systemverilog
module universal_cache_tb;

logic clk;
logic rst;
logic [7:0] key;
logic [7:0] value;
logic put;
logic [7:0] output_value;
logic hit;

universal_cache #(.KEY_WIDTH(8), .VALUE_WIDTH(8), .CACHE_SIZE(4)) dut (
    .clk(clk),
    .rst(rst),
    .key(key),
    .value(value),
    .put(put),
    .output_value(output_value),
    .hit(hit)
);

initial begin
    $dumpfile("universal_cache_tb.vcd");
    $dumpvars(0, universal_cache_tb);
end

initial begin
    clk = 0;
    forever #5 clk = ~clk;
end

initial begin
    rst = 1;
    #10 rst = 0;
    $display("=== Universal Cache Testbench Started ===");

    $display("Testing cache put/get...");
    key = 1;
    value = 10;
    put = 1;
    #10 put = 0;
    #10 key = 2;
    #10 value = 20;
    put = 1;
    #10 put = 0;
    #10 key = 1;
    #10 if (output_value != 10) $display("Error: cache get failed");
    else $display("Test cache put/get: PASSED");

    $display("Testing cache hit...");
    key = 2;
    #10 if (hit != 1) $display("Error: cache hit failed");
    else $display("Test cache hit: PASSED");

    $display("Testing cache miss...");
    key = 3;
    #10 if (hit != 0) $display("Error: cache miss failed");
    else $display("Test cache miss: PASSED");

    $display("=== Testbench Completed ===");
    #10 $finish;
end

endmodule
```

Verilator Simulation Output:
=== Universal Cache Testbench Started ===
Testing cache put/get...
Test cache put/get: PASSED
Testing cache hit...
Test cache hit: PASSED
Testing cache miss...
Test cache miss: PASSED
=== Testbench Completed ===
Process finished with return code: 0
Removing Chapter_11_examples/example_10__universal_cache/obj_dir directory...
Chapter_11_examples/example_10__universal_cache/obj_dir removed successfully.


0

## Nested Classes

### 11. Simple Car with Engine
**Description:** Car class with nested Engine class. Shows basic composition with engine start/stop functionality inside car object.

### 12. Book with Chapter
**Description:** Book class containing nested Chapter class. Demonstrates hierarchical organization of related data structures.

### 13. Computer with Components
**Description:** Computer class with nested CPU, RAM, and Storage classes. Shows complex object composition with nested classes.

### 14. Bank Account with Transaction
**Description:** BankAccount class with nested Transaction class for tracking deposits and withdrawals internally.

### 15. Simple HTTP Request
**Description:** HTTPRequest class with nested Header and Body classes. Shows network protocol modeling with nested structures.

## Copy Constructors

### 16. Student Record Copy
**Description:** Student class with copy constructor for creating identical student records. Shows proper copying of all student data.

### 17. Simple Message Copy
**Description:** Message class demonstrating copy constructor for duplicating messages with timestamps and content.

### 18. Configuration Copy
**Description:** Config class with copy constructor for duplicating system configurations safely.

### 19. Basic Transaction Copy
**Description:** Transaction class showing how to properly copy transaction data including IDs and amounts.

### 20. Simple Data Packet Copy
**Description:** DataPacket class with copy constructor for network packet duplication with header and payload copying.

## Shallow vs. Deep Copy

### 21. Address Book Example
**Description:** Person class with Address object showing difference between shallow copy (shared address) and deep copy (separate address).

### 22. Team with Players
**Description:** Team class containing Player objects. Demonstrates shallow vs deep copying of team rosters.

### 23. Shopping Cart Demo
**Description:** ShoppingCart with Item objects showing how shallow copy shares items while deep copy creates separate item lists.

### 24. File System Node
**Description:** FileNode with child nodes showing tree structure copying. Shallow copy shares subtrees, deep copy duplicates entire tree.

### 25. Simple Database Record
**Description:** DatabaseRecord with linked data showing copying implications for related records.

## Class Handles and References

### 26. Basic Handle Demo
**Description:** Simple class showing handle assignment vs object copying. Demonstrates that handles point to same object.

### 27. Handle Array Management
**Description:** Array of class handles showing how multiple handles can reference the same objects.

### 28. Null Handle Safety
**Description:** Demonstrates null handle checking and safe object access patterns.

### 29. Handle Comparison
**Description:** Shows difference between handle equality (==) and object content comparison.

### 30. Simple Object Pool
**Description:** Basic object pool implementation managing reusable objects through handles.

## Advanced Handle Management

### 31. Reference Counter
**Description:** Simple class with reference counting to track how many handles point to an object.

### 32. Object Registry
**Description:** Central registry managing all objects by handles with lookup and cleanup functionality.

### 33. Handle Lifecycle Demo
**Description:** Shows object creation, handle assignment, and cleanup in controlled sequence.

### 34. Weak Reference Pattern
**Description:** Basic weak reference implementation that doesn't prevent object cleanup.

### 35. Handle Validation
**Description:** Utility functions for validating handles before use and detecting dangling references.

## Reference Counting and Smart Pointers Pattern

### 36. Smart Pointer Basics
**Description:** Simple smart pointer class that automatically manages object lifetime through reference counting.

### 37. Shared Resource Manager
**Description:** Resource class with automatic cleanup when no more references exist.

### 38. Automatic Memory Management
**Description:** Demonstrates automatic object destruction when reference count reaches zero.

### 39. Circular Reference Detection
**Description:** Simple example showing circular reference problem and basic detection methods.

### 40. Resource Pool with Counting
**Description:** Object pool using reference counting to manage resource allocation and deallocation.

## Best Practices and Design Patterns

### 41. Simple Factory with Types
**Description:** Factory class creating different object types based on string or enum parameters.

### 42. Template Method Pattern
**Description:** Base class with template method and derived classes implementing specific steps.

### 43. Basic Builder Pattern
**Description:** Builder class for constructing complex objects step by step with method chaining.

### 44. Simple Observer Pattern
**Description:** Subject class notifying multiple observer objects of state changes.

### 45. Strategy Pattern with Generics
**Description:** Generic strategy pattern allowing different algorithms to be plugged in at runtime.

## Common Pitfalls and Debugging

### 46. Parameter vs Type Confusion
**Description:** Side-by-side examples showing parameter vs type parameterization differences.

### 47. Handle Leak Detection
**Description:** Simple techniques for detecting when objects aren't being properly cleaned up.

### 48. Copy Constructor Pitfalls
**Description:** Common mistakes in copy constructors and their correct implementations.

### 49. Nested Class Access
**Description:** Examples of correct ways to access nested class members and common access errors.

### 50. Generic Type Safety
**Description:** Demonstrates type safety benefits and common type-related errors in parameterized classes.